## Esercizio 3 - Hanks

### Consegna

- Scegliere un verbo transitivo --> **KILL**
  
- Trovare un corpus con > 1000 frasi in cui comprare un verbo scelto (usare un verbo comune) --> link to resource: https://sentence.yourdictionary.com/kill
- Effettuare parsing e disambiguazione
- Usare i supersensi di wordnet sugli argomenti (subj e obj nel caso di 2 argomenti) del verbo scelto
- Calcolo risultati, frequenza e stampare cluster semantici ottenuti

### Approccio

Partendo da un dataset di frasi con il verbo *kill* abbiamo estratto, tramite il *Dependency Matcher* di *spacy*, 
l'oggetto e il soggetto del verbo.

Dopo di che abbiamo ricavato i synset di entrambi.

Visto che il nostro dataset presentava molte frasi in cui soggeto o oggetto erano parole come *"you", "me", "someone",...*
abbiamo automaticamente associato come synset *person* a tutti i termini simili a quelli indicati precedentemente.
(La lista completa è presente nella lista "person")

Infine abbiamo stampato le statistiche del nostro dataset, indicando la percentuale di volte in cui il verbo kill comprare
con soggetto/verbo appartenenti alla stessa categoria, o meglio che si riconducono allo stesso synset.

**Come distinguo i synset?**
--> Funzione lesk (https://www.nltk.org/howto/wsd.html#word-sense-disambiguation)

**Come trovo i supersensi?**
--> Funzione lexname (https://wordnet.princeton.edu/documentation/lexnames5wn)
I Synsets in wordnet sono organizzati sotto 45 categorie, abbiamo deciso di utilizzare questa funzione per estrarre le categorie semantiche a cui associare
i soggetti e gli oggetti estratti dalle frasi

### Esecuzione

In [1]:
from nltk.corpus import wordnet
from spacy.matcher import DependencyMatcher
from nltk.wsd import lesk
import re
import spacy

* person: Usata per trovare il supersenso dei pronomi
* patter: Usato per trovare il soggetto e l'oggetto del verbo kill

In [2]:
person = ["i", "you", "he", "she", "we", "they", "me", "him", "her", "his", "them", "someone", "us", "people", "anyone"] 

pattern1 = [
    {"RIGHT_ID": "attr",
    "RIGHT_ATTRS": {"LEMMA": {"IN": ["kill"]}}
    },
    {"LEFT_ID": "attr",
    "REL_OP": ">",
    "RIGHT_ID": "subj",
    "RIGHT_ATTRS": {"DEP": {"IN": ["nsubj"]}}
    },
    {"LEFT_ID": "attr",
    "REL_OP": ">",
    "RIGHT_ID": "dobj",
    "RIGHT_ATTRS": {"DEP": {"IN": ["dobj"]}}
    }
]

pattern2 = [
    {"RIGHT_ID": "verb",
    "RIGHT_ATTRS": {"LEMMA": {"IN": ["want", "wish"]}}
    },
    {"LEFT_ID": "verb",
    "REL_OP": ">",
    "RIGHT_ID": "subj",
    "RIGHT_ATTRS": {"DEP": {"IN": ["nsubj"]}}
    },
    {"LEFT_ID": "verb",
    "REL_OP": ">",
    "RIGHT_ID": "xcomp",
    "RIGHT_ATTRS": {"DEP": {"IN": ["xcomp"]}}
    },
    {"LEFT_ID": "xcomp",
    "REL_OP": ">",
    "RIGHT_ID": "dobj",
    "RIGHT_ATTRS": {"DEP": {"IN": ["dobj"]}}
    }
]

#### Carico spacy e aggiungo il pattern al Matcher

In [3]:
nlp = spacy.load('en_core_web_sm')

matcher = DependencyMatcher(nlp.vocab)
matcher.add("pattern1", [pattern1])
matcher.add("pattern2", [pattern2])

#### Metodi per trovare il soggetto e l'oggetto del verbo e per fare word sense disambiguation

In [4]:
def get_match(text):
    # Find the pattern in the document
    doc = nlp(text)
    matches = matcher(doc)
    for match in matches:
        match_words = sorted(match[1])
        phrase = doc[match_words[0]:match_words[len(match_words)-1]+1]
        subj = phrase[0].text
        dobj = phrase[len(phrase)-1].text
        
        return subj,dobj,phrase[0].tag_,phrase[len(phrase)-1].tag_
    return "","","",""

def word_sense_disambiguation(list_words, word):
    right_synset = lesk(list_words, word)
    return right_synset

In [5]:
def cleaner(text):
    res = text.split('.')
    return res[1]

In [18]:
subj_ss = ""
dobj_ss = ""
struct = {}
tot = 0
c = 0
with open ('../data/sentence_kill.txt', 'r', encoding="utf8") as f:
    for row in f:
        subj_synset, dobj_synset, subj_ss, dobj_ss = None, None, "", ""
        subj, dobj, stag, dtag = get_match(row)
            
        if subj != "" and dobj != "":
            # Cerco il synset del soggetto e dell'oggetto e associo automaticamente il synset "person" se trovo
            # un nome proprio o una sringa presente in person
            
            # Soggetto
            if stag == "NNP" or subj.lower() in person:
                subj_ss = "person"
            else:
                subj_synset = word_sense_disambiguation(re.findall(r'\w+', row), subj)
                
            # Oggetto
            if dtag == "NNP" or dobj.lower() in person:
                dobj_ss = "person"
            else:
                dobj_synset = word_sense_disambiguation(re.findall(r'\w+', row), dobj)        
            
            # Soggetto - Se subj_synset e' None, significa che abbiamo associato il synset person
            if not subj_synset is None:
                subj_ss = cleaner(subj_synset.lexname())
            elif subj_ss != "person":
                subj_ss = "unknown"
            
            # Oggetto
            if not dobj_synset is None:
                dobj_ss = cleaner(dobj_synset.lexname())  
            elif dobj_ss != "person":
                dobj_ss = "unknown"

            if (subj_ss, dobj_ss) in struct:
                struct[(subj_ss, dobj_ss)] += 1
            else:
                struct[(subj_ss, dobj_ss)] = 1
        else:
            c+=1
            #print(f"phrase {c}: {row}")

# Used to check the number of sentences with no match
for (k,v) in zip(struct.keys(),struct.values()):
    if (k[0] != "unknown" and k[1] != "unknown") and v > 1:
        tot += v

somma_tot = 0
for k in struct.keys():
    if (k[0] != "unknown" and k[1] != "unknown") and struct[k] > 1:
        print(f"{k}: {round(((struct[k]/tot)*100), 2)} %")
        somma_tot += ((struct[k]/tot)*100)
print(f"Somma delle percentuali: {round(somma_tot, 3)} %")


('artifact', 'communication'): 1.16 %
('person', 'person'): 70.52 %
('person', 'group'): 1.73 %
('person', 'communication'): 2.89 %
('person', 'animal'): 3.47 %
('act', 'person'): 1.16 %
('person', 'all'): 5.2 %
('person', 'artifact'): 2.31 %
('cognition', 'person'): 1.73 %
('person', 'cognition'): 1.16 %
('person', 'act'): 1.73 %
('group', 'person'): 3.47 %
('contact', 'person'): 1.16 %
('person', 'emotion'): 1.16 %
('social', 'person'): 1.16 %
Somma delle percentuali: 100.0 %


### Analisi dei risultati

Le percentuali ottenute indicano la frequenza con cui kill compare con un soggetto/oggetto appartenente ad una determinata categoria.

Possiamo notare che il **70%** delle volte compare con **person** sia come oggetto che come soggetto, il che è abbastanza plausibile se 
andiamo ad analizzare il dataset che abbiamo utilizzato e come abbiamo determinato il tipo *person*.

L'approccio è abbastanza scalabile siccome il *dependency matcher* è abbastanza veloce, ma potrebbero servire altri pattern in base al dataset 
utilizzato.

Questo esperimento ha evidenziato come sia difficile disambiguare un verbo per un calcolatore, vedendo la quantità di tipi diversi che abbiamo 
individuato, ma anche come l'approccio di Patrick Hanks sia molto semplice da eseguire e produca risultati molto precisi.

**Cosa si potrebbe migliorare?**

- *Word Sense Disambiguation*: Per fare WSD e trovare il synset corretto abbiamo utilizzato la funzione lesk di wordnet che come sappiamo
  non funziona perfettamente e potrebbe restituire un synset non corretto

- *Individuazione supersensi*: Abbiamo utilizzato la funzione lexname() per individuare i supersensi degli argomenti del verbo, è possibile che utilizzando
  altri approcci si ottengano risultati diversi.

- *Dataset*: Il dataset che abbiamo utilizzato contiene poche frasi e quindi non è un campione significativo del verbo *kill*, anche se 
  nonostante ciò siamo riusciti a catturare una discreta quantità di sensi diversi.

- *Significati del verbo*: Sulla base dei risultati ottenuti si potrebbe associare un senso al verbo sulla base della categoria dell'oggetto e del soggetto,
  ad esempio *'person' kill 'person'* potrebbe rappresentare l'azione di uccidere inteso come togliere la vita ad una persona, mentre
  *'person' kill 'artifactal'* potrebbe rappresentare l'azione terminare un processo, come ad esempo *"kill a rumor"* (spegnere la luce).
